In [1]:
from keras.models import load_model
new_model = load_model('/workspaces/codespaces-jupyter/data/models/conv64x4_dense512x2_y0.1_minlr0.001.h5')

2024-01-31 20:54:09.823050: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:54:09.856211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 20:54:09.856255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 20:54:09.857541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 20:54:09.863522: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:54:09.864498: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import random
import numpy as np
import pandas as pd


class MinesweeperEnv(object):
    def __init__(self, width, height, n_mines, rewards={'win':1, 'lose':-1, 'progress':0.3, 'guess':-0.3, 'no_progress' : -0.3}):
        self.nrows, self.ncols = width, height
        self.ntiles = self.nrows * self.ncols
        self.n_mines = n_mines
        self.grid = self.init_grid()
        self.board = self.get_board()
        self.state, self.state_im = self.init_state()
        self.n_clicks = 0
        self.n_progress = 0
        self.n_wins = 0

        SIZEOFSQ = 100
        self.rewards = rewards

    def init_grid(self):
        board = np.zeros((self.nrows, self.ncols), dtype='object')
        mines = self.n_mines

        while mines > 0:
            row, col = random.randint(0, self.nrows-1), random.randint(0, self.ncols-1)
            if board[row][col] != 'B':
                board[row][col] = 'B'
                mines -= 1

        return board

    def get_neighbors(self, coord):
        x,y = coord[0], coord[1]

        neighbors = []
        for col in range(y-1, y+2):
            for row in range(x-1, x+2):
                if ((x != row or y != col) and
                    (0 <= col < self.ncols) and
                    (0 <= row < self.nrows)):
                    neighbors.append(self.grid[row,col])

        return np.array(neighbors)

    def count_bombs(self, coord):
        neighbors = self.get_neighbors(coord)
        return np.sum(neighbors=='B')

    def get_board(self):
        board = self.grid.copy()

        coords = []
        for x in range(self.nrows):
            for y in range(self.ncols):
                if self.grid[x,y] != 'B':
                    coords.append((x,y))

        for coord in coords:
            board[coord] = self.count_bombs(coord)

        return board

    def get_state_im(self, state):
        '''
        Gets the numeric image representation state of the board.
        This is what will be the input for the DQN.
        '''

        state_im = [t['value'] for t in state]
        state_im = np.reshape(state_im, (self.nrows, self.ncols, 1)).astype(object)

        state_im[state_im=='U'] = -1
        state_im[state_im=='B'] = -2

        state_im = state_im.astype(np.int8) / 8
        state_im = state_im.astype(np.float16)

        return state_im

    def init_state(self):
        unsolved_array = np.full((self.nrows, self.ncols), 'U', dtype='object')

        state = []
        for (x, y), value in np.ndenumerate(unsolved_array):
            state.append({'coord': (x, y), 'value':value})

        state_im = self.get_state_im(state)

        return state, state_im

    def color_state(self, value):
        if value == -1:
            color = 'white'
        elif value == 0:
            color = 'slategrey'
        elif value == 1:
            color = 'blue'
        elif value == 2:
            color = 'green'
        elif value == 3:
            color = 'red'
        elif value == 4:
            color = 'midnightblue'
        elif value == 5:
            color = 'brown'
        elif value == 6:
            color = 'aquamarine'
        elif value == 7:
            color = 'black'
        elif value == 8:
            color = 'silver'
        else:
            color = 'magenta'

        return f'color: {color}'

    def draw_state(self, state_im):
        state = state_im * 8.0
        state_df = pd.DataFrame(state.reshape((self.nrows, self.ncols)), dtype=np.int8)

        display(state_df.style.applymap(self.color_state))

    def click(self, action_index):
        coord = self.state[action_index]['coord']
        value = self.board[coord]

        # ensure first move is not a bomb
        if (value == 'B') and (self.n_clicks == 0):
            grid = self.grid.reshape(1, self.ntiles)
            move = np.random.choice(np.nonzero(grid!='B')[1])
            coord = self.state[move]['coord']
            value = self.board[coord]
            self.state[move]['value'] = value
        else:
            # make state equal to board at given coordinates
            self.state[action_index]['value'] = value

        # reveal all neighbors if value is 0
        if value == 0.0:
            self.reveal_neighbors(coord, clicked_tiles=[])

        self.n_clicks += 1

    def reveal_neighbors(self, coord, clicked_tiles):
        processed = clicked_tiles
        state_df = pd.DataFrame(self.state)
        x,y = coord[0], coord[1]

        neighbors = []
        for col in range(y-1, y+2):
            for row in range(x-1, x+2):
                if ((x != row or y != col) and
                    (0 <= col < self.ncols) and
                    (0 <= row < self.nrows) and
                    ((row, col) not in processed)):

                    # prevent redundancy for adjacent zeros
                    processed.append((row,col))

                    index = state_df.index[state_df['coord'] == (row,col)].tolist()[0]

                    self.state[index]['value'] = self.board[row, col]

                    # recursion in case neighbors are also 0
                    if self.board[row, col] == 0.0:
                        self.reveal_neighbors((row, col), clicked_tiles=processed)

    def reset(self):
        self.n_clicks = 0
        self.n_progress = 0
        self.grid = self.init_grid()
        self.board = self.get_board()
        self.state, self.state_im = self.init_state()

    def checkpattern(self, col, row):
        #Function to construct the checked pattern in pygame
        if row % 2:
            if col % 2: #If unequal
                return True
            else: #if equal
                return False
        else:
            if col % 2: #If unequal
                return False
            else: #if equal
                return True

    def step(self, action_index):
        done = False
        coords = self.state[action_index]['coord']

        current_state = self.state_im

        # get neighbors before action
        neighbors = self.get_neighbors(coords)

        self.click(action_index)

        # update state image
        new_state_im = self.get_state_im(self.state)
        self.state_im = new_state_im

        if self.state[action_index]['value']=='B': # if lose
            reward = self.rewards['lose']
            done = True

        elif np.sum(new_state_im==-0.125) == self.n_mines: # if win
            reward = self.rewards['win']
            done = True
            self.n_progress += 1
            self.n_wins += 1

        elif np.sum(self.state_im == -0.125) == np.sum(current_state == -0.125):
            reward = self.rewards['no_progress']

        else: # if progress
            if all(t==-0.125 for t in neighbors): # if guess (all neighbors are unsolved)
                reward = self.rewards['guess']

            else:
                reward = self.rewards['progress']
                self.n_progress += 1 # track n of non-isoloated clicks

        return self.state_im, reward, done


In [18]:

import time
mine = MinesweeperEnv(9,9,10)
mine.reset()
while True: 
    moves = new_model.predict(np.reshape(mine.state_im.reshape(9,9), (1, 9, 9, 1)))
    moves[mine.state_im.reshape(1,81)!=-0.125] = np.min(moves)
    move = np.argmax(moves)
    print(move)
    res = mine.step(move)[2]
    mine.draw_state(mine.state_im)
    if (res):
        print("Game over")
        break
    time.sleep(1)

1/1 [==============================] - 0s 14ms/step
15


/tmp/ipykernel_20973/611306781.py:48: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(neighbors=='B')


,0,1,2,3,4,5,6,7,8
0,-1,-1,-1,-1,-1,1,0,0,0
1,-1,-1,-1,-1,-1,1,0,0,0
2,-1,-1,-1,-1,2,1,0,0,0
3,-1,-1,-1,-1,1,0,1,1,1
4,-1,-1,-1,-1,2,0,1,-1,-1
5,-1,-1,-1,-1,1,1,2,-1,-1
6,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1,-1,-1


1/1 [==============================] - 0s 13ms/step
78


,0,1,2,3,4,5,6,7,8
0,-1,-1,-1,-1,-1,1,0,0,0
1,-1,-1,-1,-1,-1,1,0,0,0
2,-1,-1,-1,-1,2,1,0,0,0
3,-1,-1,-1,-1,1,0,1,1,1
4,-1,-1,-1,-1,2,0,1,-1,-1
5,-1,-1,-1,-1,1,1,2,-1,-1
6,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,2,-1,-1


...

1/1 [==============================] - 0s 14ms/step
59


,0,1,2,3,4,5,6,7,8
0,-1,1,0,1,1,1,0,0,0
1,1,1,0,1,-1,1,0,0,0
2,0,0,1,2,2,1,0,0,0
3,0,0,1,-1,1,0,1,1,1
4,0,0,2,2,2,0,1,-1,1
5,0,0,1,-1,1,1,2,2,1
6,0,1,2,2,2,2,-1,1,0
7,1,2,-1,1,-1,-1,3,1,0
8,1,-1,2,1,2,-1,2,0,0


1/1 [==============================] - 0s 14ms/step
67


,0,1,2,3,4,5,6,7,8
0,-1,1,0,1,1,1,0,0,0
1,1,1,0,1,-1,1,0,0,0
2,0,0,1,2,2,1,0,0,0
3,0,0,1,-1,1,0,1,1,1
4,0,0,2,2,2,0,1,-1,1
5,0,0,1,-1,1,1,2,2,1
6,0,1,2,2,2,2,-1,1,0
7,1,2,-1,1,2,-1,3,1,0
8,1,-1,2,1,2,-1,2,0,0


Game over
